In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
from copy import deepcopy


# 为了画图的方便, 我们只使用二维的情况
def GDA_update_step(alpha, nabla_x, nabla_y):
    """
    构造一个function, 这个function输入 x, y, 生成 x_{t+1}, y_{t+1}
    
    x_{t+1} = x_t - \alpha \nabla_x f(x_t, y_t)
    y_{t+1} = y_t + \alpha \nabla_y f(x_t, y_t)
    
    x_t, y_t are float type
    alpha is the learning rate
    nabla_x, nabla_y are the function that takes in the x_t and y_t that returns float
    
    return the next step of x and y
    """
    def update(x_t, y_t):
        x_t_plus_1 = x_t - alpha * nabla_x(x_t, y_t)
        y_t_plus_1 = y_t + alpha * nabla_y(x_t, y_t)
        return x_t_plus_1, y_t_plus_1
    return update


def OGDA_update_step(alpha, nabla_x, nabla_y):
    """
    构造一个function, 这个function输入 x, y, 生成 x_{t+1}, y_{t+1}
    
    x_{t+1} = x_t - 2*\alpha \nabla_x f(x_t, y_t) + \alpha \nabla_x f(x_{t-1}, y_{t-1})
    y_{t+1} = y_t + 2*\alpha \nabla_y f(x_t, y_t) - \alpha \nabla_y f(x_{t-1}, y_{t-1})
    
    x_t, y_t are float type
    alpha is the learning rate
    nabla_x, nabla_y are the function that takes in the x_t and y_t that returns float
    
    return the next step of x and y
    
    
    Special Attention: For the first step, we use the GDA update rule
    Thus we only need to let x_t_minus_1 = x_t, y_t_minus_1 = y_t
    """
    def update(x_t, y_t, x_t_minus_1, y_t_minus_1):
        x_t_plus_1 = x_t - 2 * alpha * nabla_x(x_t, y_t) + alpha * nabla_x(x_t_minus_1, y_t_minus_1)
        y_t_plus_1 = y_t + 2 * alpha * nabla_y(x_t, y_t) - alpha * nabla_y(x_t_minus_1, y_t_minus_1)
        return x_t_plus_1, y_t_plus_1
    return update



from tqdm import tqdm
def get_timestep_images(x_0_lis, y_0_lis, update_function, steps, images_folder_path, kernel="GDA"):
    """
    This function takes in the initial x and y of many states, the update function, the number of steps and the folder path to save the images
    x_0, y_0: float
    update_function: GDA_update_step or OGDA_update_step
    steps: int, the number of steps
    images_folder_path: str, the folder path to save the images
    """
    x_current_lis, y_current_list = x_0_lis, y_0_lis
    # x_lis[i][j] is the x of the j-th state at the i-th step
    x_lis = [x_current_lis.copy()]
    y_lis = [y_current_list.copy()]
    import os
    os.makedirs(images_folder_path, exist_ok=True)
    

    if kernel == "GDA":
        print('GDA updating---')
        for _ in tqdm(range(steps)):
            for i in range(len(x_current_lis)):
                x_current_lis[i], y_current_list[i] = update_function(x_current_lis[i], y_current_list[i])
            x_lis.append(x_current_lis.copy())
            y_lis.append(y_current_list.copy())
    elif kernel == "OGDA":
        """
        OGDA 第一步的更新公式使用 GDA 的更新公式
        OGDA 的更新公式为
        x_{t+1} = x_t - \alpha \nabla_x f(x_t, y_t)
        y_{t+1} = y_t + \alpha \nabla_y f(x_t, y_t)
        """
        pass

		# 第一步先用 GDA 的更新公式
        for i in range(len(x_current_lis)):
            x_current_lis[i], y_current_list[i] = update_function(x_t = x_current_lis[i], 
                                                                  y_t = y_current_list[i],
                                                                  x_t_minus_1 = x_current_lis[i],
                                                                  y_t_minus_1 = y_current_list[i])
        x_lis.append(x_current_lis.copy())
        y_lis.append(y_current_list.copy())
        print('OGDA_updating---')
        for _ in tqdm(range(steps - 1)):
            for i in range(len(x_current_lis)):
                x_current_lis[i], y_current_list[i] = update_function(x_t = x_current_lis[i], 
																	  y_t = y_current_list[i],
                                                                      x_t_minus_1 = deepcopy( x_lis[-2][i]),
                                                                      y_t_minus_1 = deepcopy( y_lis[-2][i]))
																	#   x_t_minus_1 = x_lis[-2][i].copy(),
																	#   y_t_minus_1 = y_lis[-2][i].copy())
            x_lis.append(x_current_lis.copy())
            y_lis.append(y_current_list.copy())

    



    x_min, x_max = min(min(x_lis)), max(max(x_lis))
    y_min, y_max = min(min(y_lis)), max(max(y_lis))
    for i in tqdm(range(steps + 1)):
        plt.figure(figsize=(10, 10))
        for j in range(len(x_0_lis)):
            plt.plot(x_lis[i][j], y_lis[i][j], 'ro')
        # TODO(BobHunagC): there no need to draw all the trajectory, for normal step, just keep for 100 steps
        for _state_idx in range(len(x_0_lis)):
            if i != steps:
                _tmp_draw_x_lis = [x_lis[_time_idx][_state_idx] for _time_idx in range(max(0, i-100), i+1)]
                _tmp_draw_y_lis = [y_lis[_time_idx][_state_idx] for _time_idx in range(max(0, i-100), i+1)]
            elif i == steps:
                _tmp_draw_x_lis = [x_lis[_time_idx][_state_idx] for _time_idx in range(0, i+1)]
                _tmp_draw_y_lis = [y_lis[_time_idx][_state_idx] for _time_idx in range(0, i+1)]			
            plt.plot(_tmp_draw_x_lis, _tmp_draw_y_lis, 'bo-', markersize=0.1)

        plt.xlim(x_min - 0.1, x_max + 0.1)
        plt.ylim(y_min - 0.1, y_max + 0.1)
        plt.title(f'Step {i}')
        plt.savefig(f'{images_folder_path}/step_{i}.png')
        plt.close()

	


def gif_generation(images_num, images_folder_path, fps=10, step=5):
    """
    生成GIF
    images_num: 图像的数量
	images_folder_path: 图像所在文件夹的路径
    """
    images = []
    for i in range(0, images_num, step):
        images.append(imageio.imread(f'{images_folder_path}/step_{i}.png'))
    imageio.mimsave(f'{images_folder_path}-optimization.gif', images, fps=fps)



In [ ]:
# # GDA test
# # f(x, y) = x^2 + y
# # nabla_x = 2x, nabla_y = 1
# test_GDA_update_step = GDA_update_step(alpha = 0.1, nabla_x= lambda x, y: 2 * x, nabla_y=lambda x, y: 1)

# get_timestep_images(x_0_lis = [10, 10, 7, 8], y_0_lis = [0, 1, 2, 3], update_function=test_GDA_update_step, steps=100, images_folder_path='images_test')

# gif_generation(images_num=100, 
#                images_folder_path='images_test')



In [5]:
# # OGDA test
# # f(x, y) = x^2 + y
# # nabla_x = 2x, nabla_y = 1
test_GDA_update_step = OGDA_update_step(alpha = 0.1, nabla_x= lambda x, y: 2 * x, nabla_y=lambda x, y: 1)

get_timestep_images(x_0_lis = [10, 10, 7, 8], 
                    y_0_lis = [0, 1, 2, 3], 
                    update_function=test_GDA_update_step, 
                    steps=200, 
                    images_folder_path='images_test',
                    kernel="OGDA")

gif_generation(images_num=100, 
               images_folder_path='images_test')



100%|██████████| 201/201 [00:28<00:00,  6.96it/s]
/tmp/ipykernel_56672/2735699990.py:143: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f'{images_folder_path}/step_{i}.png'))


In [2]:
# figure 1
# f(x, y) = -1/8 x^2 - 1/2 y^2 + 6/10 xy
# nabla_x = -1/4 x + 6/10 y, nabla_y = -y + 6/10 x

nabla_x = lambda x, y: -1/4 * x + 6/10 * y
nabla_y = lambda x, y: -y + 6/10 * x
alpha = 0.01
x_0_lis = np.linspace(-0.25, 0.25, num=10)
y_0_lis = np.linspace(-0.25, 0.25, num=10) 
x_0_lis = list(x_0_lis) * 10
y_0_lis = list(y_0_lis) * 10
y_0_lis = sorted(y_0_lis)

steps = 1000
images_folder_path = 'figure_1'


In [ ]:

test_GDA_update_step = GDA_update_step(alpha = alpha, 
                                       nabla_x= nabla_x,
                                        nabla_y=nabla_y)

get_timestep_images(x_0_lis = x_0_lis, 
                    y_0_lis = y_0_lis, 
                    update_function=test_GDA_update_step, 
                    steps=steps,
                    images_folder_path=images_folder_path)

In [3]:
gif_generation(images_num=800,
                images_folder_path=images_folder_path,
                fps=steps//10, 
                step=10)


/tmp/ipykernel_60219/2735699990.py:143: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f'{images_folder_path}/step_{i}.png'))
